In [1]:
from nltk.tokenize import word_tokenize
import jieba
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# import nltk
# nltk.download('punkt')
 

Using TensorFlow backend.


In [2]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [3]:
import keras

In [4]:
from gensim.models import KeyedVectors

In [5]:
en_path = './Gensim/Word2Vec/GoogleNews-vectors-negative300.bin'
# ch_path = './Gensim/Word2Vec/udn.word2vec.bin'

en_wv = KeyedVectors.load_word2vec_format(en_path, binary=True)
# ch_wv = KeyedVectors.load_word2vec_format(ch_path, binary=True)

en_word = en_wv.wv
# ch_word = ch_wv.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [6]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [7]:
text = open('./cmn.txt').readlines()

In [8]:
input_texts = []
target_texts = []
target_characters = set()

for line in text:
  
    input_text, target_text = line.strip().split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
#   for word in input_text.split():
#       if word not in input_characters:
#           input_characters.add(char)
    for word in list(jieba.cut(target_text)):
        if word not in target_characters:
            target_characters.add(word)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpbqwg172e' -> '/tmp/jieba.cache'
Loading model cost 0.790 seconds.
Prefix dict has been built succesfully.


In [9]:
max_encoder_seq_length = max([len(word_tokenize(txt)) for txt in input_texts])
num_encoder_tokens = 300

target_characters = sorted(list(target_characters))
num_decoder_tokens = len(target_characters)
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [12]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(word_tokenize(input_text)):
        if word in en_word:
            encoder_input_data[i, t] = en_wv[word]
    #       else:
    #           UNKNOWN WORD    
    #       encoder_input_data[i, t, input_token_index[char]] = 1.
#     for t, char in enumerate(target_text):
    for t, char in enumerate(list(jieba.cut(target_text))):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [13]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('./nlplab03.h5')

Train on 16106 samples, validate on 4027 samples
Epoch 1/100
16106/16106 [==============================] - 103s 6ms/step - loss: 0.8501 - val_loss: 1.3550
Epoch 2/100
16106/16106 [==============================] - 85s 5ms/step - loss: 0.7468 - val_loss: 1.2568
Epoch 3/100
16106/16106 [==============================] - 85s 5ms/step - loss: 0.6925 - val_loss: 1.2452
Epoch 4/100
16106/16106 [==============================] - 86s 5ms/step - loss: 0.6578 - val_loss: 1.2002
Epoch 5/100
16106/16106 [==============================] - 85s 5ms/step - loss: 0.6244 - val_loss: 1.1755
Epoch 6/100
16106/16106 [==============================] - 86s 5ms/step - loss: 0.5944 - val_loss: 1.1491
Epoch 7/100
16106/16106 [==============================] - 86s 5ms/step - loss: 0.5687 - val_loss: 1.1344
Epoch 8/100
16106/16106 [==============================] - 86s 5ms/step - loss: 0.5459 - val_loss: 1.1281
Epoch 9/100
16106/16106 [==============================] - 85s 5ms/step - loss: 0.5252 - val_loss: 1.1

16106/16106 [==============================] - 108s 7ms/step - loss: 0.1367 - val_loss: 1.3268
Epoch 78/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1352 - val_loss: 1.3331
Epoch 79/100
16106/16106 [==============================] - 109s 7ms/step - loss: 0.1338 - val_loss: 1.3281
Epoch 80/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1320 - val_loss: 1.3268
Epoch 81/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1310 - val_loss: 1.3327
Epoch 82/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1297 - val_loss: 1.3349
Epoch 83/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1283 - val_loss: 1.3327
Epoch 84/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1270 - val_loss: 1.3412
Epoch 85/100
16106/16106 [==============================] - 108s 7ms/step - loss: 0.1257 - val_loss: 1.3354
Epoch 86/100
16106/16106 [===============

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [15]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
# reverse_input_char_index = dict(
#     (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [16]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [17]:

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Run.
Decoded sentence: 你用跑。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I try.
Decoded sentence: 让我来。

-
Input sentence: I won!
Decoded sentence: 我赢了。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 你!

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 去过。

-
Input sentence: I lost.
Decoded sentence: 我10了您。

-
Input sentence: I quit.
Decoded sentence: 我讨厌。

-
Input sentence: I'm OK.
Decoded sentence: 我沒事。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 不可能！

-
Input sentence: No way!
Decoded sentence: 不可能！

-
Input sentence: Really?
Decoded sentence: 你确定？

-
Input sentence: Try it.
Decoded sentence: 试试吧。

-
Input sentence: We try.
Decoded sentence: 我们来试试。

-
Input sentence: Why